In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from zipfile import ZipFile
from tqdm.auto import tqdm
import os
import argparse

In [2]:
# making the directories
os.system('rm -rf data && mkdir data')
os.system('rm -rf data/Mild && mkdir data/Mild')
os.system('rm -rf data/Severe && mkdir data/Severe')
os.system('rm -rf data/No_DR && mkdir data/No_DR')
os.system('rm -rf data/Moderate && mkdir data/Moderate')
os.system('rm -rf data/Proliferate && mkdir data/Proliferate')

os.system('rm -rf testdata && mkdir testdata')
os.system('rm -rf testdata/Mild && mkdir testdata/Mild')
os.system('rm -rf testdata/Severe && mkdir testdata/Severe')
os.system('rm -rf testdata/No_DR && mkdir testdata/No_DR')
os.system('rm -rf testdata/Moderate && mkdir testdata/Moderate')
os.system('rm -rf testdata/Proliferate && mkdir testdata/Proliferate')
# if you have more than 2 classes then make the directories as per use

0

In [ ]:
# class ResidualBlock(nn.Module):
#     def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
#         super(ResidualBlock, self).__init__()
#         self.conv1 = nn.Sequential(
#                         nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
#                         nn.BatchNorm2d(out_channels),
#                         nn.ReLU())
#         self.conv2 = nn.Sequential(
#                         nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
#                         nn.BatchNorm2d(out_channels))
#         self.downsample = downsample
#         self.relu = nn.ReLU()
#         self.out_channels = out_channels
        
#     def forward(self, x):
#         residual = x
#         out = self.conv1(x)
#         out = self.conv2(out)
#         if self.downsample:
#             residual = self.downsample(x)
#         out += residual
#         out = self.relu(out)
#         return out

In [ ]:


# class ResNet(nn.Module):
#     def __init__(self, block, layers, num_classes = 10):
#         super(ResNet, self).__init__()
#         self.inplanes = 64
#         self.conv1 = nn.Sequential(
#                         nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3),
#                         nn.BatchNorm2d(64),
#                         nn.ReLU())
#         self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
#         self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
#         self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
#         self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
#         self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
#         self.avgpool = nn.AvgPool2d(7, stride=1)
#         self.fc = nn.Linear(512, num_classes)
        
#     def _make_layer(self, block, planes, blocks, stride=1):
#         downsample = None
#         if stride != 1 or self.inplanes != planes:
            
#             downsample = nn.Sequential(
#                 nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
#                 nn.BatchNorm2d(planes),
#             )
#         layers = []
#         layers.append(block(self.inplanes, planes, stride, downsample))
#         self.inplanes = planes
#         for i in range(1, blocks):
#             layers.append(block(self.inplanes, planes))

#         return nn.Sequential(*layers)
    
    
#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.maxpool(x)
#         x = self.layer0(x)
#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.layer3(x)

#         x = self.avgpool(x)
#         x = x.view(x.size(0), -1)
#         print(x.shape)
#         x = self.fc(x)

#         return x

In [ ]:
# num_classes = 10
# num_epochs = 20
# batch_size = 16
# learning_rate = 0.01

# net = ResNet(ResidualBlock, [3, 4, 6, 3])

# # Loss and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9) 

In [3]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, padding='same')
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3, padding='same')
        self.fc1 = nn.Linear(50176, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 5)

    def forward(self, x):
        #print(x.shape)
        x = self.pool(F.relu(self.conv1(x)))
        #print(x.shape)
        x = self.pool(F.relu(self.conv2(x)))
        #print(x.shape)
        x = torch.flatten(x, 1)
        #print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
# this is the model

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4901, 0.4617, 0.4061], [0.1977, 0.1956, 0.1947])
])

data = torchvision.datasets.ImageFolder(
    root='data',
    transform=transform
)

classes = data.classes
dataloader = torch.utils.data.DataLoader(data, batch_size=32, shuffle=True, num_workers=1)

In [6]:
if os.path.exists('model.pt'):
    print('----> loaded pretrained model')
    net.load_state_dict(torch.load('model.pt'))

In [ ]:
num_epochs = 30

print(f'----> starting training for {num_epochs} epochs')

with tqdm(range(num_epochs), desc='training') as training_bar:
    for epoch in training_bar:
        running_loss = 0.0
        epoch_bar = tqdm(enumerate(dataloader), desc=f'epoch {epoch + 1}')
        for i, data in epoch_bar:
            inputs, labels = data
            outputs = net(inputs)

            optimizer.zero_grad()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_bar.set_postfix(average_running_loss=running_loss / (i + 1))

print('----> finished training')


In [8]:
testdata = torchvision.datasets.ImageFolder(
    root='testdata',
    transform=transform
)
testdataloader = torch.utils.data.DataLoader(testdata, batch_size=32, shuffle=True, num_workers=1)

In [10]:
correct = 0
total = 0
with torch.no_grad():
    for testdata in testdataloader:
        inputs, labels = testdata
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'----> accuracy on the dataset: {100 * correct / total:.2f}%')

----> accuracy on the dataset: 66.67%


In [11]:
print('----> saving model to model.pt')

torch.save(net.state_dict(), 'model.pt')

----> saving model to model.pt
